In [ ]:
import pandas as pd
import time as t
from lxml import html
import requests
import csv

reviews_df = pd.DataFrame()

mainpageindex = 0


while mainpageindex != 5:
    print(mainpageindex)
    searchlink= 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Concord%2C+CA&ns=1&start=' +str(mainpageindex*10)

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36'

    headers = {'User-Agent': user_agent}

    page = requests.get(searchlink, headers = headers)
    parser = html.fromstring(page.content)

    businesslink=parser.xpath('//a[@class=" link__09f24__1kwXV link-color--inherit__09f24__3PYlA link-size--inherit__09f24__2Uj95"]')
    links = [l.get('href') for l in businesslink]

    u = []
    for link in links:
        if str(link).find('adredir') == -1 :  #ignore sponsored listings
            u.append('https://www.yelp.com'+ str(link))
            #t.sleep(10)
    i=1
    for item in u:
        print("Item " + str(i) + ": " + item )
        page = requests.get(item, headers = headers)
        print(page)
        parser = html.fromstring(page.content)
        i+=1
        xpath_reviews = '//li[@class="lemon--li__373c0__1r9wz margin-b5__373c0__2ErL8 border-color--default__373c0__3-ifU"]'
        reviews = parser.xpath(xpath_reviews)
        print(reviews)
        t.sleep(10)
        print(len(reviews))
        for review in reviews:
            print("Review Count:" + str(reviewcount))
            temp = review.xpath('.//div[contains(@class, "lemon--div__373c0__1mboc i-stars__373c0__1T6rz i-stars--regular-5__373c0__N5JxY border-color--default__373c0__3-ifU overflow--hidden__373c0__2y4YK")]')
      
            rating = [td.get('aria-label') for td in temp]
            print(rating)
            xpath_body    = './/span[@class="lemon--span__373c0__3997G raw__373c0__3rcx7"]//text()'
    
            body    = review.xpath(xpath_body)
            print(body)

            review_dict = { 'rating': rating, 
                'review': body,
              }
            reviews_df = reviews_df.append(review_dict, ignore_index=True)
            reviewcount += 1
            t.sleep(10)
    mainpageindex += 1


print(reviews_df)



page_nums = '//a[@class="lemon--a__373c0__IEZFH link__373c0__1G70M pagination-link-component__373c0__1fUdr link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE"]'
pg = parser.xpath(page_nums)
max_pg=len(pg)+1

print(max_pg)
    
    
fields = ['rating','review']

filename = "YelpDataset1000.csv"

print(reviews_df.size)
print(reviews_df)

reviews_df.to_csv(filename,index=False)
